In [1]:
import getpass
import pandas as pd
import requests
import json

Bouw de $lookup Bundle adhv input spreadsheet met codes+system

In [4]:
df = pd.read_excel("codelijsten.xls", "Blad2")


In [47]:
payload = {
   "type" : "batch",
   "resourceType" : "Bundle",
   "entry" : [
   ]
}

done = []
for row in df.iterrows():
    # row[1] is a Panda Series, convert to a list array
    _cells = row[1].to_list()
    _code = _cells[1]
    _system = _cells[2]
    # only do if not already done
    if (not(_code in done)):
        done.append(_code)
        entry = {
             "resource" : {
                "parameter" : [
                   {
                      "name" : "system",
                      "valueUri" : _system
                   },
                   {
                      "name" : "code",
                      "valueCode" : _code
                   }
                ],
                "resourceType" : "Parameters"
             },
             "request" : {
                "method" : "POST",
                "url" : "CodeSystem/$lookup"
             }
          }
        payload["entry"].append(entry)
        
print (json.dumps(payload))

{"type": "batch", "resourceType": "Bundle", "entry": [{"resource": {"parameter": [{"name": "system", "valueUri": "http://snomed.info/sct"}, {"name": "code", "valueCode": 419511003}], "resourceType": "Parameters"}, "request": {"method": "POST", "url": "CodeSystem/$lookup"}}, {"resource": {"parameter": [{"name": "system", "valueUri": "http://snomed.info/sct"}, {"name": "code", "valueCode": 418471000}], "resourceType": "Parameters"}, "request": {"method": "POST", "url": "CodeSystem/$lookup"}}, {"resource": {"parameter": [{"name": "system", "valueUri": "http://snomed.info/sct"}, {"name": "code", "valueCode": 426232007}], "resourceType": "Parameters"}, "request": {"method": "POST", "url": "CodeSystem/$lookup"}}, {"resource": {"parameter": [{"name": "system", "valueUri": "http://snomed.info/sct"}, {"name": "code", "valueCode": 419199007}], "resourceType": "Parameters"}, "request": {"method": "POST", "url": "CodeSystem/$lookup"}}, {"resource": {"parameter": [{"name": "system", "valueUri": "ht

Gebruik nu de NTS om de $lookup Bundle uit te voeren

In [20]:
username=input("NTS username: ")
password = getpass.getpass("NTS password: ")

NTS username: m.van.der.zel@umcg.nl
NTS password: ········


In [38]:
url = "https://terminologieserver.nl/auth/realms/nictiz/protocol/openid-connect/token"

apayload='username='+username+'&password='+password+'&grant_type=password&client_id=cli_client'
headers = { 'Content-Type': 'application/x-www-form-urlencoded' }

response = requests.request("POST", url, headers=headers, data=apayload)

token = response.json()
access_token = token["access_token"]

In [48]:
url = "https://terminologieserver.nl/fhir"

payloadstring = json.dumps(payload)

headers = { 'Authorization': 'Bearer ' + access_token,
          'Content-Type': 'application/fhir+json'}

response = requests.request("POST", url, headers=headers, data=payloadstring)

print(response.text)

{"resourceType":"Bundle","id":"36c2c5ce-70d3-4aa3-bfbc-24f4819a47f8","type":"batch-response","link":[{"relation":"self","url":"https://terminologieserver.nl/fhir"}],"entry":[{"resource":{"resourceType":"Parameters","parameter":[{"name":"name","valueString":"module van Nederlandse editie"},{"name":"version","valueString":"http://snomed.info/sct/11000146104/version/20210930"},{"name":"display","valueString":"neiging tot ongewenste reactie op medicatie en/of drug"},{"name":"property","part":[{"name":"code","valueCode":"parent"},{"name":"value","valueCode":"418038007"}]},{"name":"property","part":[{"name":"code","valueCode":"child"},{"name":"value","valueCode":"419869001"}]},{"name":"property","part":[{"name":"code","valueCode":"child"},{"name":"value","valueCode":"418895007"}]},{"name":"property","part":[{"name":"code","valueCode":"child"},{"name":"value","valueCode":"1078301000112109"}]},{"name":"property","part":[{"name":"code","valueCode":"child"},{"name":"value","valueCode":"419787005

Gebruik response om designations op te halen en zet die in de spreadsheet terug

In [40]:
lookups = response.json()

In [49]:
rownr = 0
for row in df.iterrows():
    #row[1] is a Panda Series, convert to a list array
    _cells = row[1].to_list()
    _code = _cells[1]
    _system = _cells[2]
    if (lookups["entry"][rownr]["resource"]["resourceType"] == "Parameters"):
        entryparams = lookups["entry"][rownr]["resource"]["parameter"]
        display = [x for x in entryparams if x["name"] == "display"]
        print(_code, ",", display[0]["valueString"])
   #     designations = [x for x in entryparams if x["name"] == "designation" and x["part"][0]["valueCode"] == "nl"]
   #     if (designations):
   #         print(rownr, _code, designations[0]["part"][2])
   #     else:
   #         print(rownr, _code, "geen nl designations")
    else:
        print(_code, ",", "? niet gevonden")
    rownr = rownr + 1

419511003 , neiging tot ongewenste reactie op medicatie en/of drug
418471000 , neiging tot ongewenste reactie op voedsel
426232007 , omgevingsgerelateerde allergie
426232007 , omgevingsgerelateerde allergie
426232007 , omgevingsgerelateerde allergie
419199007 , allergie voor substantie
419511003 , neiging tot ongewenste reactie op medicatie en/of drug
419511003 , neiging tot ongewenste reactie op medicatie en/of drug
789-8 , ? niet gevonden
2951-2 , ? niet gevonden
2951-2 , ? niet gevonden
2951-2 , ? niet gevonden
2951-2 , ? niet gevonden
2951-2 , ? niet gevonden
2951-2 , ? niet gevonden
2951-2 , ? niet gevonden
2951-2 , ? niet gevonden
2951-2 , ? niet gevonden
2951-2 , ? niet gevonden
2951-2 , ? niet gevonden
2951-2 , ? niet gevonden
2951-2 , ? niet gevonden
2951-2 , ? niet gevonden
2951-2 , ? niet gevonden
26881-3 , ? niet gevonden
74103-3 , ? niet gevonden
62357-9 , ? niet gevonden
32637-1 , ? niet gevonden
